In [ ]:
using GRIB

Y = 2017

#Temperature data of the year of interest from ERA5-Land in .grib format
infilename = "GRIB_files/Spain_$(Y).grib" 

#Filename to save the daily GDD output data
outfilename = "GDD_data/$(Y)/PS_GDD"

T_base =  9.2 + 273.15 #9.2 + 273.15 #9 + 273.15
T_opt = 23.4 + 273.15 #27.6 + 273.15 #27.4 + 273.15
T_max = 34.2 + 273.15 #41.8 + 273.15 #41.2 + 273.15

#Compute GDD by month
@time GribFile(infilename) do f

    m = Message(f)
    
    GDD = zeros(size(vec(m["values"])))
    
    current_day = 1
    
    message_prev = m
    
    for message in f

        T = vec(message["values"])
        day = Int(message["day"])
        
        #write GDD for this month
        if day != current_day
            
            date = string(current_day, "_", message_prev["month"], "_", message_prev["year"])
            
            println("Writing ", date)
           
            lats = message["latitudes"]
            lons = message["longitudes"]
            
            shape = (size(message["values"])[2], size(message["values"])[1])
            
            file = open(string(outfilename, "_", date, ".txt"), "w")
            
            println(file, "#GDD\tLongitudes\tLatitudes\tshape=$shape")
            
            for i in 1 : length(GDD)
                
                println(file, GDD[i] / 24.0, "\t", round(lons[i], digits=4), "\t", round(lats[i], digits=4))
                
            end
            
            close(file)
            
            current_day = day
            GDD = zeros(size(vec(message["values"])))
            
        end
        
        A = - (T_opt - T_base) / (T_max - T_opt)
        B = -A * T_max

        for i in eachindex(T)

                if T[i] < T_base
               
                    #Nothing            

                elseif T[i] <= T_opt
                    
                    GDD[i] += (T[i] - T_base)
                
                elseif T[i] <= T_max
            
                    GDD[i] += (A*T[i] + B)
                
                end

        end
        
        message_prev = message
        
    end
    
    current_day = 31
    
    date = string(current_day, "_", message_prev["month"], "_", message_prev["year"])
            
    println("Writing ", date)

    lats = m["latitudes"]
    lons = m["longitudes"]

    shape = (size(m["values"])[2], size(m["values"])[1])

    file = open(string(outfilename, "_", date, ".txt"), "w")

    println(file, "#GDD\tLongitudes\tLatitudes\tshape=$shape")

    for i in 1 : length(GDD)

        println(file, GDD[i] / 24.0, "\t", round(lons[i], digits=4), "\t", round(lats[i], digits=4))

    end

    close(file)
    
end